In [1]:
# this code is from https://github.com/pumpikano/tf-dann, without any change

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tarfile
import os
import pickle as pkl
import numpy as np
import skimage
import skimage.io
import skimage.transform
from tensorflow.examples.tutorials.mnist import input_data

# load MNIST data from tensorflow library
mnist = input_data.read_data_sets('MNIST_data')

# name of data.zip
BST_PATH = 'BSR_bsds500.tgz'

# number of methods for generating random numbers to call by class
rand = np.random.RandomState(42)

# open the zip or tar
f = tarfile.open(BST_PATH)

# storing the name files from training data
train_files = []
for name in f.getnames():
    if name.startswith('BSR/BSDS500/data/images/train/'):
        train_files.append(name)

print('Loading BSR training images')

# extract a member from archieve
background_data = []
for name in train_files:
    try:
        fp = f.extractfile(name)        # extract file
        bg_img = skimage.io.imread(fp)  # load an image from file
        background_data.append(bg_img)  # save data into list
    except:
        continue


def compose_image(digit, background):
    """Difference-blend a digit and a random patch from a background image."""
    w, h, _ = background.shape
    dw, dh, _ = digit.shape
    x = np.random.randint(0, w - dw)
    y = np.random.randint(0, h - dh)

    bg = background[x:x + dw, y:y + dh]
    return np.abs(bg - digit).astype(np.uint8)


def mnist_to_img(x):
    """Binarize MNIST digit and convert to RGB."""
    x = (x > 0).astype(np.float32)
    d = x.reshape([28, 28, 1]) * 255
    return np.concatenate([d, d, d], 2)


def create_mnistm(X):
    """
    Give an array of MNIST digits, blend random background patches to
    build the MNIST-M dataset as described in
    http://jmlr.org/papers/volume17/15-239/15-239.pdf
    """
    X_ = np.zeros([X.shape[0], 28, 28, 3], np.uint8)
    for i in range(X.shape[0]):

        if i % 1000 == 0:
            print('Processing example', i)

        bg_img = rand.choice(background_data)

        d = mnist_to_img(X[i])
        d = compose_image(d, bg_img)
        X_[i] = d

    return X_


print('Building train set...')
train = create_mnistm(mnist.train.images)
print('Building test set...')
test = create_mnistm(mnist.test.images)
print('Building validation set...')
valid = create_mnistm(mnist.validation.images)

# Save dataset as pickle
with open('mnistm_data.pkl', 'wb') as f:
    pkl.dump({'train': train, 'test': test, 'valid': valid}, f, pkl.HIGHEST_PROTOCOL)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Loading BSR training images
Building train set...
Processing example 0
Processing example 1000
Processing example 2000
Processing example 3000
Processing example 4000
Processing example 5000
Processing example 6000
Processing example 7000
Processing example 8000
Processing example 9000
Processing example 10000
Processing example 11000
Processing example 12000
Processing example 13000
Processing example 14000
Processing example 15000
Processing example 16000
Processing example 17000
Processing example 18000
Processing example 19000
Processing e